In [1]:
import json
import requests
import requests_cache
import pandas as pd
import pprint
import numpy as np
import isodbtools

requests_cache.install_cache('api_map_cache')

# Download Helpful Data Files

In [2]:
# Build a Library of Adsorptives
#   NOTE: this will accelerate name <-> InChIKey mapping
URL = 'https://adsorption.nist.gov/isodb/api/gases.json'
adsorptives = requests.get(URL).json()

# Build a Library of Adsorbents
URL = 'https://adsorption.nist.gov/matdb/api/materials.json'
adsorbents = requests.get(URL).json()

# Download the (abbreviated) Isotherm Library
URL = 'https://adsorption.nist.gov/isodb/api/isotherms.json'
isotherms = requests.get(URL).json()

# Define Search Criteria

In [3]:
# Adsorptive Species
adsorptive = 'CO2'
adsorptive = requests.get('https://adsorption.nist.gov/isodb/api/gas/'+adsorptive+'.json').json()
pprint.pprint(adsorptive)
target_InChIKey = adsorptive['InChIKey']

# Temperature Range
temperature_min = 273 #K
temperature_max = 323 #K

{'InChICode': 'InChI=1S/CO2/c2-1-3',
 'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N',
 'formula': 'CO2',
 'name': 'Carbon Dioxide',
 'synonyms': ['Carbon dioxide', 'Carbon oxide', 'CO2']}


# Filter Isotherms Library against first-level criteria

In [4]:
candidate_isotherms = []

for index,isotherm in enumerate(isotherms):
    # Select single-component isotherms with desired adsorptive
    if len(isotherm['adsorbates']) == 1 and isotherm['adsorbates'][0]['InChIKey'] == target_InChIKey:
        if temperature_min <= isotherm['temperature'] <= temperature_max:
            #print(isotherm['filename'], isotherm['temperature'])
            candidate_isotherms.append(isotherm['filename'])
    #if index == 1000: break

# Filter Isotherms against second-level criteria

In [5]:
Rg = 8.314462618  #J/mol K  = kPa l / mol K  CODATA 2018 [exact]

p_atm = 1.01325 #bar
T_atm = 298 #K
y_CO2 = 400. / 1.e6  # using 400 ppm CO2
p_CO2_atm = y_CO2 * p_atm  # partial pressure of CO2 at atmospheric conditions

In [ ]:
from scipy.interpolate import *

def interpolate_loading(pressure,adsorption,p_interp):
    # Add the origin if it's not already there
    if min(pressure) != 0.:
        pressure = [0.] + pressure
        adsorption = [0.] + adsorption
    if min(pressure) < 0.:
        raise Exception('ERROR: negative pressure encountered')
    # Use a spline to interpolate the isotherm
    ads_spline = splrep(pressure,adsorption)
    ads_interp = splev(p_interp,ads_spline)
    return ads_interp    

filtered_isotherms_dict = []

for index,filename in enumerate(candidate_isotherms):
    URL = 'https://adsorption.nist.gov/isodb/api/isotherm/'+filename+'.json&k=dontrackmeplease'
    isotherm_full = requests.get(URL).json()
    #pprint.pprint(isotherm_full)
    T_iso = isotherm_full["temperature"]
    p_CO2_iso = p_CO2_atm * T_iso / T_atm  # equivalent pressure for isotherm temperature
    q_units = isotherm_full['adsorptionUnits']
    #print(p_CO2_iso, p_CO2_atm)
    
    p = [x['pressure'] for x in isotherm_full['isotherm_data']]
    q = [x['species_data'][0]['adsorption'] for x in isotherm_full['isotherm_data']]
    try:
        loading_estimate = interpolate_loading(p,q,p_CO2_iso)
        #print( isotherm_full['adsorbent']['name'], isotherm_full['temperature'], 'K',
        #       p_CO2_iso, 'bar', loading_estimate, q_units)
        if loading_estimate > 0:
            filtered_isotherms_dict.append({'DOI' : isotherm_full['DOI'],
                                            'name': isotherm_full['adsorbent']['name'],
                                            'hashkey': isotherm_full['adsorbent']['hashkey'],
                                            'T': isotherm_full['temperature'],
                                            'p': p_CO2_iso,
                                            'q': loading_estimate,
                                            'q_units': isotherm_full['adsorptionUnits'],
                                            'filename': isotherm_full['filename']
                                           })
        else:
            print('negative adsorption encountered', filename)
        
    except:
        print('negative pressure encountered', filename)
    
    if index == 500: break
    if (index+1)%50 == 0: print('finished ', index)

# Turn the nasty results in to a nice Pandas Dataframe
filtered_isotherms = pd.DataFrame(
    list(zip(
            [x['DOI'] for x in filtered_isotherms_dict], 
            [x['filename'] for x in filtered_isotherms_dict],
            [x['name'] for x in filtered_isotherms_dict],
            [x['T'] for x in filtered_isotherms_dict],
            [x['p'] for x in filtered_isotherms_dict],
            [x['q'] for x in filtered_isotherms_dict],
            [x['q_units'] for x in filtered_isotherms_dict],
            )),
    columns =['DOI',
              'filename',
              'Material Name',
              'Temperature (K)',
              'Equivalent CO2 Pressure (bar)',
              'Loading Estimate',
              'Loading Units'
             ]
)

#filtered_isotherms

filtered_isotherms.to_csv('co2_loading.csv')

negative adsorption encountered 10.1002adfm.201101479.Isotherm15
negative adsorption encountered 10.1002adfm.201101479.Isotherm16
negative adsorption encountered 10.1002adfm.201101479.Isotherm18
negative adsorption encountered 10.1002adfm.201101479.Isotherm6
negative adsorption encountered 10.1002adfm.201101479.Isotherm7
negative adsorption encountered 10.1002adfm.201101479.Isotherm8
negative adsorption encountered 10.1002adfm.201502069.Isotherm6
negative adsorption encountered 10.1002adfm.201502069.Isotherm7
negative adsorption encountered 10.1002adma.200602645.Isotherm1
negative adsorption encountered 10.1002adma.201403827.Isotherm9
finished  49
negative adsorption encountered 10.1002adsu.201700092.Isotherm13
negative adsorption encountered 10.1002aic.10687.Isotherm26
negative adsorption encountered 10.1002Aic.11865.isotherm3
negative adsorption encountered 10.1002Aic.13970.isotherm3
negative adsorption encountered 10.1002Aic.13970.isotherm4
negative adsorption encountered 10.1002Aic

In [35]:
filtered_isotherms

,DOI,filename,Material Name,Temperature (K),Equivalent CO2 Pressure (bar),Loading Estimate,Loading Units
0,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm13,Zn(BDC)(DMBPY) 0.5,288,0.000392,0.006969307897361737,wt%
1,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm14,Zn(BDC)(DMBPY) 0.5,298,0.000405,0.00207280383509775,wt%
2,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm17,Zn(NDC)(DMBPY) 0.5,288,0.000392,0.0005393593582411948,wt%
3,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm19,Zn(BDC)(BPY) 0.5,288,0.000392,0.0024484150622842028,wt%
4,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm20,Zn(BDC)(BPY) 0.5,298,0.000405,0.002423222461167524,wt%
...,...,...,...,...,...,...,...
72,10.1002/Aic.14467,10.1002Aic.14467.Isotherm24,Zn-MOF-74,313,0.000426,0.0016531898187895572,mmol/g
73,10.1002/Aic.14467,10.1002Aic.14467.Isotherm25,CuBTC,313,0.000426,0.0038672161158793505,mmol/g
74,10.1002/Aic.14467,10.1002Aic.14467.Isotherm26,Mg-MOF-74,313,0.000426,0.009873859577237664,mmol/g
75,10.1002/Aic.14467,10.1002Aic.14467.Isotherm6,IRMOF-1,298,0.000405,0.0019660431570271693,mmol/g
